In [1]:
from utils import set_root

import pandas as pd
import numpy as np

import os
import seaborn as sns
import matplotlib.pyplot as plt

set_root()

Current working directory: /home/guido/shared/github/physioex-private


In [6]:
results_path = "models/multi-source-domain/"
combinations = list( range(0, 2) ) # 0, 1

In [7]:
comb_results = {}
for combo in combinations:
    
    comb_results[combo] = []
    
    comb_dir = os.path.join(results_path, f"k={combo}")
    # take all the files in the directory or subdirectories which are named results.csv
    
    results = []
    for root, dirs, files in os.walk(comb_dir):
        for file in files:
            if str(file) == "results.csv":
                results.append( os.path.join(root, file) )
        
    for result in results:    
        
        result_df = pd.read_csv( result )
        # take only the test_results
        result_df = result_df[ result_df["split"] == "test" ]
        result_df = result_df.drop(columns=["split", "dataset"]).mean()
        result_df = pd.DataFrame(result_df).T
                
        comb_results[combo].append( result_df )

print( comb_results[0] )

print( pd.concat( comb_results[1] ).mean() )

[   test_loss  test_acc   test_f1   test_ck   test_pr   test_rc
0   1.176623  0.650994  0.614425  0.499358  0.669822  0.650994]
test_loss    1.261752
test_acc     0.607999
test_f1      0.583928
test_ck      0.448682
test_pr      0.653703
test_rc      0.607999
dtype: float64


results with k=8